In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
import keras as keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
# Test GPU availability
tf.config.list_physical_devices('GPU')

In [ ]:
img_path = '/home/robotics20/robotic_arena_ws/ROS/src/cameras_ctrl/robot_captures/'
npy_path = '/home/robotics20/robotic_arena_ws/ROS/src/cameras_ctrl/robot_captures_npy/'
recimg_path = '/home/robotics20/robotic_arena_ws/ROS/src/dac_modules/recimgs/'
model_path = '/home/robotics20/robotic_arena_ws/ROS/src/dac_modules/model/autoencoder_offline.h5'

In [ ]:
# Load .npy dataset
npys = os.listdir(npy_path)
data = np.array([np.load(npy_path + npy) for npy in npys])
print(data.shape)

In [ ]:
def build_model(img_shape=(84,84,3)):  
    input_img = Input(shape=img_shape)
    filter_size = (3, 3)
    pooling_size = (2, 2)
    
    x = Conv2D(16, filter_size, activation='relu', padding='same')(input_img)
    x = MaxPooling2D(pooling_size, padding='same')(x)
    x = Conv2D(32, filter_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pooling_size, padding='same')(x)
    x = Conv2D(32, filter_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pooling_size, padding='same')(x)
    
    x = Flatten()(x)
    x = Dense(20, activation='relu')(x)
    x = Dense(11*11*32)(x)
    x = Reshape((11, 11, 32))(x)

    x = Conv2D(32, filter_size, activation='relu', padding='same')(x)
    x = UpSampling2D(pooling_size)(x)
    x = Conv2D(32, filter_size, activation='relu', padding='same')(x)
    x = UpSampling2D(pooling_size)(x)
    x = Conv2D(16, filter_size, activation='relu')(x)
    x = UpSampling2D(pooling_size)(x)
    x = Conv2D(3, filter_size, activation='relu', padding='same')(x)

    model = Model(input_img, x) 
    optimizer = RMSprop(learning_rate=0.0005)  
    model.compile(loss='mse', optimizer=optimizer) 
    model.summary() 
    model.save(model_path)
    
    return model

In [ ]:
def train(model, data, epochs, batch_size):
    history = model.fit(data, data, epochs=epochs, batch_size=batch_size)
    #error = history.history['loss'][-1]
    return history

def save_image(prototype, indx):
    rec_img = prototype[0]
    rec_img = cv2.cvtColor(rec_img, cv2.COLOR_RGB2BGR)
    img_name = recimg_path + str(indx) + '.png'
    cv2.imwrite(img_name, rec_img)
    
def predict(model, img, indx):
    #m = Model(inputs=model.inputs, outputs=model.get_layer('dense').output)
    #prototype = m.predict(np.expand_dims(img, axis=0))
    prototype = model.predict(img.reshape((1,84,84,3)))
    save_image(prototype, indx)
    return prototype

In [ ]:
# Only run this cell to build the initial autoencoder
autoencoder = build_model()

In [ ]:
# Train the autoencoder
autoencoder = keras.models.load_model(model_path)
history = train(autoencoder, data, 5000, 16)
autoencoder.save(model_path)

In [ ]:
# Plot errors
plt.plot(history.history['loss'])

In [ ]:
# Save the predicted/reconstructed images
autoencoder = keras.models.load_model(model_path)
for npy in npys:
    predict(autoencoder, np.load(npy_path + npy), npy.replace('.npy', ''))

In [ ]:
# Vision inputs vs. reconstructed images
indx = 835

#plt.imshow(mpimg.imread(img_path + str(indx) + '.png'))
plt.imshow(np.load(npy_path + str(indx) + '.npy'))
plt.show()

plt.imshow(mpimg.imread(recimg_path + str(indx) + '.png'))
#plt.imshow(np.load(recimg_path + str(indx) + '.npy'))
plt.show()